In [1]:
from __future__ import print_function
import lmdb
import caffe
import numpy as np

__author__ = 'PedroMorgado'

In [2]:
def load_lmdb(fn):
    env = lmdb.open(fn, readonly=True)
    datum = caffe.proto.caffe_pb2.Datum()
    with env.begin() as txn:
        cursor = txn.cursor()
        data, labels = [], []
        for _, value in cursor:
            datum.ParseFromString(value)
            labels.append(datum.label)
            data.append(caffe.io.datum_to_array(datum).squeeze())
    env.close()
    return data, labels

In [3]:
def prep_image(img):
    img = img.astype(float)[:, 14:241, 14:241]             # center crop (img is in shape [C, X, Y])
    img -= np.array([104., 117., 123.]).reshape((3,1,1))     # demean (same as in trainval.prototxt
    return img

In [4]:
gpu_id = 1
images, labels = load_lmdb('test.LMDB')

caffe.set_mode_gpu()
caffe.set_device(gpu_id)
deploy = caffe.Net('caffenet/deploy.prototxt', caffe.TEST, weights='model.caffemodel')
probs = []
for i in range(0, len(images), 25):
    batch = [prep_image(img) for img in images[i:i+25]]
    batch_size = len(batch)

    deploy.blobs['data'].data[:batch_size] = batch
    deploy.forward()
    # Note np.copy. Otherwise, next forward() step will replace memory
    probs.append(np.copy(deploy.blobs['prob'].data[:batch_size, :]))
probs = np.concatenate(probs, 0)
print('Accuracy', (probs.argmax(1)==np.array(labels)).mean()*100)

Accuracy 100.0
